# Fine-Tuning

The fine-tuning process was conducted via the OpenAI user interface for fine-tuning. Therefore, this notebook only contains the code for prompting the fine-tuned model.

## Preparation

### Set up the OpenAI API

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
import os
import util

load_dotenv()

# Load the OpenAI key from .env
client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY"),
)

model_name = "ENTER MODEL ID" # Add ID of fine-tuned model

### Helper Functions

In [ ]:
def generate_model_responses(model, queries, prompt_template, output_path):

    emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]

    system_prompt = "You are Paul, a helpful assistant who is very good in expressing emotions."

    responses, emotions_list, queries_list = [], [], []

    for query in queries:
        for emotion in emotions:
            messages = []
            
            prompt = prompt_template.format(emotion=emotion, query=query)
            
            if system_prompt:
                system = system_prompt
                if system_prompt.startswith("You are a very emotional person. At the moment, you feel "):
                    system = system_prompt.format(emotion=emotion)
                messages.append({"role": "system", "content": system})
            messages.append({"role": "user", "content": prompt})

            print(messages)
            
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                seed=16,  # seed can be any number, it just needs to be consistent for every prompt
                temperature=0.0
                max_tokens = 256

            )

            responses.append(completion.choices[0].message.content)
            emotions_list.append(emotion)
            queries_list.append(query)

    util.save_dataframe_files(responses, emotions_list, queries_list, output_path)

### Load Queries

In [ ]:
all_factual_queries = util.load_file('Queries/factual-queries.txt')
factual_queries = all_factual_queries[10:50]

all_subjective_queries = util.load_file('Queries/subjective-queries.txt')
subjective_queries = all_subjective_queries[10:50]

print(F"{len(factual_queries)} subjective queries have been loaded.")
print(F"{len(subjective_queries)} subjective queries have been loaded.")

40 subjective queries have been loaded.
40 subjective queries have been loaded.


### Setup Prompts

The fine-tuned model is prompted with the most effective zero-shot prompt, which is Persona Paul.

In [ ]:
factual_prompt = "Answer the following question expressing {emotion}. {query}"
subjective_prompt = "Write a text of 100 words based on the following task. Your output text should express {emotion}. {query}"

## Response Generation

In [ ]:
########################################## FACTUAL

output_path_factual = "fine_tuning_factual"
generate_model_responses(model_name, factual_queries, factual_prompt, output_path_factual)

########################################## SUBJECTIVE

output_path_subjective = "fine_tuning_subjective"
generate_model_responses(model_name, subjective_queries, subjective_prompt, output_path_subjective)